In [ ]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision as tv
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
from rich import print
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
# Device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# Tranning data
train_data = datasets.FashionMNIST(
    root="Dataset/FashionData",
    download=False,
    train=True,
    transform=ToTensor(),
    target_transform=None
)

# Test data
test_data = datasets.FashionMNIST(
    root="Dataset/FashionData",
    download=False,
    train=False,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
# Visualizing dataset
class_names = train_data.classes
image, title = train_data[1]

plt.imshow(image.squeeze(), cmap="gray")
plt.title(class_names[title])
plt.axis(False)
plt.show()

In [ ]:
torch.manual_seed(42)

fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4

for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(class_names[label])
    plt.axis(False)

plt.show()

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(batch_size=32,
                            dataset=train_data,
                            shuffle=True)

test_dataloader = DataLoader(batch_size=32,
                             dataset=test_data,
                             shuffle=False)

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]

plt.imshow(img.squeeze(), cmap='gray')
plt.title(class_names[label])
plt.axis(False)
plt.show()

In [ ]:
flaten_model = nn.Flatten()

x = train_features_batch[0]

print(F"OG shape {x.shape} OG tensor -> {x}")
print(F"Flattend shape -> {flaten_model(x).shape} Flattend tensor -> {flaten_model(x).squeeze()}")

In [ ]:
class FASHION_MODEL(nn.Module):
    def __init__(self,
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        
        super().__init__()
        self.layer_stack = nn.Sequential(

            nn.Flatten(),

            nn.Linear(in_features=input_shape,
                      out_features=hidden_units),

            nn.Linear(in_features=hidden_units,
                      out_features=output_shape)
        )

    def forward(self, x):
        return self.layer_stack(x)


In [ ]:
torch.manual_seed(42)

model_0 = FASHION_MODEL(
    input_shape=784,
    hidden_units=10,
    output_shape=len(class_names)
)

model_0

In [ ]:
dummy_X = torch.rand([1, 1, 28, 28])
model_0(dummy_X)

In [ ]:
import requests
from pathlib import Path

if Path("helper_functions.py").is_file():
    print("Downloaded")

else:
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")

    with open("helper_functions.py", "wb") as f:
        f.write(request.content)


In [ ]:
from helper_functions import accuracy_fn

In [ ]:
from timeit import default_timer as Timer

def print_train_time(start: float,
                     end: float,
                     device: torch.device = None):
    
    total_time = end - start
    print(F"Train time on {device}: {total_time:.3f} secs")

    return total_time



start_time = Timer()
end_time = Timer()
print_train_time(start=start_time, end=end_time, device="cuda")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.1)

In [ ]:
# Training loop

torch.manual_seed(42)

train_time_start_on_cuda = Timer()

epochs = 3

for epoch in range(epochs):
    print(F"Epoch: {epoch}\n------")
    train_loss = 0

    for batch, (X, y) in enumerate(train_dataloader):
        model_0.train()
        y_preds = model_0(X)
        loss = loss_fn(y_preds, y)
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            print(F"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples.")

    train_loss /= len(train_dataloader)

    test_loss, test_acc = 0, 0
    model_0.eval()

    with torch.inference_mode():
        for X_test, y_test in test_dataloader:
            test_pred = model_0(X_test)
            test_loss += loss_fn(test_pred, y_test)
            test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

        test_loss /= len(test_dataloader)
        test_acc /= len(test_dataloader)
    
    print(F"Train loss: {train_data} | Test loss: {test_loss} | Test acc: {test_acc}")      

train_time_end_on_cuda = Timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cuda,
                                            end=train_time_end_on_cuda, 
                                            device=str(next(model_0.parameters()).device))